In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions 
import requests, urllib
from bs4 import BeautifulSoup
import os, sys
from random import randint
from time import sleep

### scraping course subcategory list

In [2]:
udemy_course_page = 'https://www.udemy.com/courses/'

In [4]:
browser = webdriver.Firefox()
browser.get(udemy_course_page)

soup = BeautifulSoup(browser.page_source)

course_category_listing = []

for li in soup.find(attrs={'class':'dropdown-menu-list'}).findChildren('li', recursive=False):
    category = li.find('h4').contents[0]
    for lili in li.findAll('li'):
        subcategory_entry = {}
        subcategory_entry['category'] = category
        
        subcategory = lili.find('a').find(attrs={'class':'cat-title'}).contents[0]
        subcategory_entry['subcategory'] = subcategory
        
        subcategory_link = lili.find('a')['href']
        subcategory_entry['subcategory_link'] = subcategory_link

        print subcategory_entry
        course_category_listing.append(subcategory_entry)

In [46]:
course_category_listing = pd.DataFrame(course_category_listing)
course_category_listing.to_csv('Udemy_course_category_listing.csv', index=False)

In [2]:
course_category_listing = pd.read_csv('Udemy_course_category_listing.csv')

### Scraping course list from each subcategory

In [ ]:
class wait_for_page_load(object):

    def __init__(self, browser):
        self.browser = browser

    def __enter__(self):
        self.old_page = self.browser.find_element_by_tag_name('html')

    def page_has_loaded(self):
        new_page = self.browser.find_element_by_tag_name('html')
        return new_page.id != self.old_page.id

    def __exit__(self, *_):
        wait_for(self.page_has_loaded)

In [ ]:
with wait_for_page_load(browser):
    browser.find_element_by_link_text('my link').click()

In [47]:
courses_df = pd.read_csv('Udemy_course_list.csv', encoding='utf-8')


In [2]:
courses_df.subcategory.unique()

NameError: name 'courses_df' is not defined

In [57]:
courses_df = pd.read_csv('Udemy_course_list.csv', encoding='utf-8')
courses = []

for entry in course_category_listing.iterrows():
    if entry[1]['subcategory'] in courses_df.subcategory.unique():
        continue
    
    print entry[1]['subcategory']
    sys.stdout.flush()
    
    
    browser2 = webdriver.Firefox()
    browser2.implicitly_wait(20)
    browser2.get(entry[1]['subcategory_link']+'All-Courses/')

    first_page = True

    sleep(10)
#     WebDriverWait(browser2, 1000).until(expected_conditions.presence_of_element_located(browser2.find_element_by_class_name('num')))
    soup = BeautifulSoup(browser2.page_source)
    try:
        total_pages = soup.findAll( attrs={'class':'num'})[-1].find( attrs={'class':'skin-color'}).contents[0]
    except:
        total_pages = 1
    total_pages = int(total_pages)
    
    try:
        for page in xrange(1, total_pages+1):
            if not first_page:
                a = browser2.find_elements_by_class_name('arr')[1]
                b = a.find_element_by_class_name('skin-color')
                b = a.find_element_by_class_name('skin-color').click()
                soup = BeautifulSoup(browser2.page_source)

            for course in soup.findAll(attrs={'class':'course-box-flat'}):
                first_page = False
                course_info = {}
                course_info['subcategory'] = entry[1]['subcategory']
                course_info['course_url'] = course.find('a')['href']
                course_info['course_name'] = course.find(attrs={'class':'title'}).contents[0]
                try:
                    course_info['course_price'] = course.find(attrs={'class':'price'}).contents[0]
                except:
                    course_info['course_price'] = 'na'
                course_info['course_rev_count'] = course.find(attrs={'class':'rev-count'}).contents[0]
                course_info['course_level'] = course.find(attrs={'class':'course-det'}).contents[0]
                course_info['course_rating'] = course.find( attrs={'class':'s-rating'}).contents[1]['style']

                courses.append(course_info)

        courses_df_additional = pd.DataFrame(courses)
    #     courses_df = pd.DataFrame(courses)
        courses_df = courses_df.append(courses_df_additional)
        courses_df.to_csv('Udemy_course_list.csv', index=False, encoding='utf-8')

        browser2.close()
    except:
        continue

Personal Finance
Career Development
Parenting & Relationships
Happiness
Religion & Spirituality
Personal Brand Building
Creativity
Influence
Self Esteem
Stress Management
Memory & Study Skills
Motivation
Web Design
Graphic Design
Design Tools
User Experience
Game Design
Design Thinking
3D & Animation
Fashion
Architectural Design
Interior Design
Digital Marketing
Search Engine Optimization
Social Media Marketing
Branding
Marketing Fundamentals
Analytics & Automation
Public Relations
Advertising
Video & Mobile Marketing
Content Marketing
Non-Digital Marketing
Growth Hacking
Affiliate Marketing
Product Marketing
Arts & Crafts
Food & Beverage
Beauty & Makeup
Travel
Gaming
Home Improvement
Pet Care & Training
Digital Photography
Photography Fundamentals
Portraits
Landscape
Black & White
Photography Tools
Mobile Photography
Travel Photography
Commercial Photography
Wedding Photography
Wildlife Photography
Video Design
Fitness
General Health
Sports
Nutrition
Yoga
Mental Health
Dieting
Self De

In [2]:
courses_df = pd.read_csv('Udemy_course_list.csv')

In [3]:
courses_df_dup_dropped = courses_df.drop_duplicates()

In [4]:
courses_df_dup_dropped.shape  ## need to debug the code for why missing courses... 

(7603, 7)

In [6]:
print courses_df_dup_dropped.loc[courses_df_dup_dropped.subcategory == 'Humanities'].shape
courses_df_dup_dropped.loc[courses_df_dup_dropped.subcategory == 'Humanities'].course_name

(48, 7)


443708                 Paul and His Letter to the Galatians
443709                                     Simply Good News
443710    Chinese Made Easy L2: Understand 79% of Chines...
443711    Write a Novel Outline from Scratch (Novel Writ...
443712              Worldviews, the Bible, and the Believer
443713    Critical Thinker Academy: Learn to Think Like ...
443714                                     Creative Writing
443715     Write Now: Creative Writing Skills for Beginners
443716                                        Paraphrasing 
443717     Creative Writing – Entering Writing Competitions
443718               Creative Writing - Writing Tips part 1
443719    Fortune Favors the Brave: Ancient Lessons for ...
443756                                  Ethics: An Overview
443757    You Can Teach Your Child to Read Very Successf...
443758                        20th-century American History
443759                    Why are you alive? - Introduction
443760           The Interface Between S

#### scraping course description

##### course description

In [5]:
def extract_text_from_Udemy_Course_page_DOM_element(soup, attr_name, attr_value):
    a = soup.find( attrs={attr_name:attr_value})

    a2 = a.findAll('p') + a.findAll('li') + a.findAll('ul')

    text = ''
    for item in a2:
        try:
            text = text + ' ' + item.contents[0]
        except:
            pass
    
    return text

In [6]:
def extract_course_info_from_Udemy_course_page(course_url):
    course_url = 'https://www.udemy.com' + course_url

    browser3 = webdriver.Firefox()
    browser3.set_window_size(5,10)
    browser3.get(course_url)

    soup = BeautifulSoup(browser3.page_source)
    course = {}

    course['rate_count'] = soup.find( attrs={'class':'rate-count'}).contents[0]
    try:
        course['course_des'] = extract_text_from_Udemy_Course_page_DOM_element(soup, 'class', 'js-simple-collapse-inner')
    except:
        course['course_des'] = 'n/a'
    try:
        course['requirements'] = extract_text_from_Udemy_Course_page_DOM_element(soup, 'id','requirements')
    except:
        course['requirements'] = 'n/a'
    try:
        course['what_you_get'] = extract_text_from_Udemy_Course_page_DOM_element(soup, 'id','what-you-get')
    except:
        course['what_you_get'] = 'n/a'
    try:
        course['who_should_attend'] = extract_text_from_Udemy_Course_page_DOM_element(soup, 'id','who-should-attend')
    except:
        course['who_should_attend'] = 'n/a'
    try:
        course['curriculum'] = extract_text_from_Udemy_Course_page_DOM_element(soup, 'id','curriculum')
    except:
        course['curriculum'] = 'n/a'
        
    browser3.close()
    
    return course

In [10]:
course_des_df = pd.read_csv('Udemy_course_description.csv', encoding='utf-8')

In [9]:
course_des_df_dup_dropped = course_des_df.drop_duplicates()

In [13]:
course_des_df.shape

(1248, 7)

In [11]:
course_des_df_dup_dropped.to_csv('Udemy_course_description.csv', encoding='utf-8')

In [10]:
course_des_df_dup_dropped.shape

(2231, 8)

In [14]:
course_des_df.to_csv('Udemy_course_description.csv', encoding='utf-8')

In [163]:
course_des_df.drop(['Unnamed: 0', u'Unnamed: 0.1'], inplace=True, axis=1)

In [164]:
course_des_df.to_csv('Udemy_course_description.csv', encoding='utf-8', index=False)

In [26]:
course_des_df = pd.read_csv('Udemy_course_description.csv', encoding='utf-8')
courses_des = []

for i, (course_name, course_url) in enumerate(zip(courses_df_dup_dropped.course_name, courses_df_dup_dropped.course_url)):
    if course_name not in course_des_df.course_name.unique():
        try:
            if (i+1) % 200 == 0:
                print i
                sys.stdout.flush()
                course_des_df_2 = pd.DataFrame(courses_des)
                course_des_df = course_des_df.append(course_des_df_2, ignore_index=True)
                course_des_df.to_csv('Udemy_course_description.csv', encoding='utf-8', index=False)

            course = extract_course_info_from_Udemy_course_page(course_url)
            course['course_name'] = course_name

            courses_des.append(course)
        except:
            continue
    else:
        continue


399
2199
2399
2599
2799
2999
3199
3399
3599
3799
3999
4199
4399
4599
4799
4999
5199
5399
5599
5799
5999
6199
6399
6599
6799
6999
7199
7399
7599


/Users/DerekZH/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [25]:
cn = course_des_df_dup_dropped.course_name.iloc[randint(0,2000)]
print cn
course_des_df_dup_dropped.loc[course_des_df_dup_dropped.course_name == cn].course_des.values



Make Money by Day Trading in Stocks


array([ u'   \t    There are lots of courses in the market that talks about stock trading in general. Here is one that talks about \u201cDay Trading\u201d and generating income as a Day Trader. This is the 2   \t    I will walk you through the day trading strategies that include the concepts of:     \t      How I prepare my gap list every day     \t    Selecting stocks to trade     \t    Managing position size and Risks         \t    Power Bar Continuation     \t    Power Bar with Pull Back     \t    Mid Bar Continuation     \t    Weak Bar Reversal     \t    Fade Play     \t    Bounce on Support Play     \t    Bounce on 200 Day Moving Average     \t    Entry and Exit Rules       \t    When Not to Trade     \t    Hard Stop vs. Soft Stop             \t      All the other topics are covered in the Part 1 series  '], dtype=object)

In [27]:
course_des_df.course_name.unique().shape

(2126,)

In [19]:
course_des_df_dup_dropped.loc[course_des_df_dup_dropped.course_name == cn]

,Unnamed: 0,course_des,course_name,curriculum,rate_count,requirements,what_you_get,who_should_attend
493,1833,PhoneGap is a modern and easy way to build Ap...,Cross-Platform Apps in 1 Hour: PhoneGap 2015 U...,NaN,\n 6 rating...,MAC or PC \n,Over 23 lectures and 1 hour of content! How t...,Anyone Interest in Building Apps \n


In [8]:
len(course_des_df.course_name.unique())

2225

In [9]:
courses_des_df = pd.DataFrame(courses_des)

In [13]:
courses_des_df

,course_des,course_name,curriculum,rate_count,requirements,what_you_get,who_should_attend
0,\t This course will provide you with...,EverWeb,\tThis video is a quick introudction to what...,3 ratings...,Mac OS X 10.6+ \n,Over 33 lectures and 1 hour of content! Build...,Complete beginners to advanced webmasters Sma...
1,If you are like one of the thousands of other...,Learn To Set Up Your Own WordPress Website; St...,,2 ratings...,Students should be familiar with using a comp...,Over 12 lectures and 47 mins of content! Purc...,This course is for beginners who are interest...
2,"In this Jquery Mobile Tutorial, you will lear...",Learning jQuery Mobile for Beginners,"In this initial lesson, I cover the experienc...",23 rating...,Experience with HTML. No prior jQuery or Java...,Over 84 lectures and 6.5 hours of content! Ba...,This course is created for both designers and...
3,In this course we´re gonna build an eye catch...,"Single Page Design with Bootstrap, CSS, HTML5,...",\tIn this video i´m gonna show what we´re go...,7 ratings...,Sublime Text 2 a Local Server (But since we´r...,Over 54 lectures and 4 hours of content! Crea...,If you´re comfortable or you´re OK with codin...
4,This class teaches you all of the essential c...,Drupal 7 for Beginners,Session Notes: 1) Login - using the credentia...,22 rating...,n/a,Over 37 lectures and 2.5 hours of content! \n,n/a
5,"New Content Added July 9, 2015 Take advantage...",Wordpress 2015 For Beginners: No Coding Build ...,\tThis video shows how to install Wordpress ...,5 ratings...,Wordpress on PC/Mac \n,Over 15 lectures and 42 mins of content! Lear...,Anyone wanting to learn Wordpress This Course...
6,\t The video course begins with a brief i...,RESTful Services with ASP.NET Web API,...,5 ratings...,This video tutorial is structured to help you...,Over 34 lectures and 2 hours of content! Unde...,This video tutorial is for web developers wit...
7,My personal love for Web2py grew from the fac...,Fun and creative web engineering with Python a...,\tIn this video I will walk through how to d...,13 rating...,Basic computer skills. Such as surfing the we...,Over 25 lectures and 2 hours of content! Afte...,This course is ment for total beginners up to...
8,\tIn this part of his series on HTML5 and C...,Learn Dynamic HTML5 Programming,,1 ratings...,PC or Laptop Text Editor \n,Over 26 lectures and 3 hours of content! Audi...,People who want to know advanced knowledge ab...
9,In this tutorial you will learn complete CSS ...,CSS and CSS3 for Beginners,,3 ratings...,html basics knowledge \n,Over 25 lectures and 1.5 hours of content! un...,Who are willing to start learn css who are in...
